In [54]:
!pip install -r requirements.txt

In [55]:
import warnings

warnings.filterwarnings("ignore")

In [56]:
import random
import torch
from transformers import BertTokenizer, BertModel
import pandas as pd

In [57]:
seed = 777
random.seed(seed)

torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

In [58]:
import torch
 
print(torch.cuda.is_available())

True


In [59]:
import torch

print(torch.cuda.is_available())
print(f"CUDA version: {torch.version.cuda}")

# Storing ID of current CUDA device
cuda_id = torch.cuda.current_device()
print(torch.cuda.current_device())
	
print(torch.cuda.get_device_name(cuda_id))


True
CUDA version: 11.8
0
NVIDIA GeForce RTX 3050 Laptop GPU


In [60]:
model_name = "neuralmind/bert-base-portuguese-cased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

In [61]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = model.to(device)

In [62]:
df = pd.read_csv('datasets/DfCleaned.csv')

df.head()

,Unnamed: 0,resp_text,clarity,Class,tokens,lemma,ngram2,ngram3
0,0,Prezado Sr Jose Taunai Em atenção ao seu pe...,c5,2,"['prezado', 'sr', 'jose', 'taunai', 'em', 'ate...",prezar sr jose taunai em atenção a o seu pedir...,"[('prezar', 'sr'), ('sr', 'jose'), ('jose', 't...","[('prezar', 'sr', 'jose'), ('sr', 'jose', 'tau..."
1,1,"""A pedido do Pró-Reitor de Graduação, informa...",c5,2,"['a', 'pedido', 'do', 'próreitor', 'de', 'grad...",o pedir de o próreitor de graduação informamo ...,"[('o', 'pedir'), ('pedir', 'de'), ('de', 'o'),...","[('o', 'pedir', 'de'), ('pedir', 'de', 'o'), (..."
2,2,"""Prezado (a) Sr. (a), Agradecemos o contato e...",c234,1,"['prezado', 'a', 'sr', 'a', 'agradecemos', 'o'...",prezar o sr o agradecer o contato e informamo ...,"[('prezar', 'o'), ('o', 'sr'), ('sr', 'o'), ('...","[('prezar', 'o', 'sr'), ('o', 'sr', 'o'), ('sr..."
3,3,"""Prezado (a) Sr. (a), Agradecemos o contato e...",c234,1,"['prezado', 'a', 'sr', 'a', 'agradecemos', 'o'...",prezar o sr o agradecer o contato e informamo ...,"[('prezar', 'o'), ('o', 'sr'), ('sr', 'o'), ('...","[('prezar', 'o', 'sr'), ('o', 'sr', 'o'), ('sr..."
4,4,"""Prezado Prof. Gilberto Tadeu Reis da Silva ...",c234,1,"['prezado', 'prof', 'gilberto', 'tadeu', 'reis...",prezar prof gilberto tader rei de o Silva em a...,"[('prezar', 'prof'), ('prof', 'gilberto'), ('g...","[('prezar', 'prof', 'gilberto'), ('prof', 'gil..."


In [63]:
def get_bert_embeddings(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512, add_special_tokens = True).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1).squeeze().cpu().numpy()
    r = list(embeddings)
    return r

In [64]:
#df['bert_embeddings'] = df['resp_text'].apply(get_bert_embeddings)
#df.head()

In [65]:
embeddings = []
for text in df['resp_text']:
    embeddings.append(get_bert_embeddings(text))

In [66]:
labels = ['emb_' + str(i) for i in range(len(embeddings[0]))]

In [67]:
df_embeddings = pd.DataFrame(embeddings, columns=labels)

In [68]:
from sklearn.preprocessing import MinMaxScaler 
import numpy as np 

In [69]:
from sklearn import preprocessing

x = df_embeddings.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df_embeddings = pd.DataFrame(x_scaled)

In [70]:
df_embeddings.head()

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,0.366699,0.436405,0.597397,0.373641,0.257723,0.675852,0.290759,0.500709,0.338782,0.468826,...,0.646290,0.461819,0.457743,0.168369,0.451732,0.390573,0.582483,0.394189,0.513412,0.569378
1,0.357845,0.643556,0.584826,0.224671,0.322027,0.532861,0.393085,0.546882,0.364420,0.315898,...,0.540465,0.347499,0.333893,0.188837,0.211751,0.342873,0.642443,0.489474,0.557652,0.561903
2,0.444821,0.499724,0.634349,0.405609,0.347355,0.520599,0.341498,0.399472,0.383273,0.415257,...,0.413470,0.404744,0.352644,0.272323,0.292353,0.365345,0.646133,0.497469,0.474224,0.546584
3,0.338893,0.664219,0.520465,0.433406,0.325693,0.413236,0.285352,0.434378,0.322330,0.492441,...,0.502520,0.439282,0.362558,0.296149,0.379228,0.237873,0.573954,0.540782,0.604582,0.450783
4,0.496814,0.420793,0.574704,0.395315,0.284557,0.593564,0.416742,0.573463,0.420579,0.451871,...,0.602760,0.576759,0.383231,0.367842,0.531815,0.299715,0.646647,0.398757,0.508204,0.586519


In [71]:
df_embeddings.shape

(6000, 768)

In [72]:
final = pd.concat([df, df_embeddings], axis=1)

In [73]:
final = final.iloc[:,1:]

In [74]:
final.iloc[0:3000]

,resp_text,clarity,Class,tokens,lemma,ngram2,ngram3,0,1,2,...,758,759,760,761,762,763,764,765,766,767
0,Prezado Sr Jose Taunai Em atenção ao seu pe...,c5,2,"['prezado', 'sr', 'jose', 'taunai', 'em', 'ate...",prezar sr jose taunai em atenção a o seu pedir...,"[('prezar', 'sr'), ('sr', 'jose'), ('jose', 't...","[('prezar', 'sr', 'jose'), ('sr', 'jose', 'tau...",0.366699,0.436405,0.597397,...,0.646290,0.461819,0.457743,0.168369,0.451732,0.390573,0.582483,0.394189,0.513412,0.569378
1,"""A pedido do Pró-Reitor de Graduação, informa...",c5,2,"['a', 'pedido', 'do', 'próreitor', 'de', 'grad...",o pedir de o próreitor de graduação informamo ...,"[('o', 'pedir'), ('pedir', 'de'), ('de', 'o'),...","[('o', 'pedir', 'de'), ('pedir', 'de', 'o'), (...",0.357845,0.643556,0.584826,...,0.540465,0.347499,0.333893,0.188837,0.211751,0.342873,0.642443,0.489474,0.557652,0.561903
2,"""Prezado (a) Sr. (a), Agradecemos o contato e...",c234,1,"['prezado', 'a', 'sr', 'a', 'agradecemos', 'o'...",prezar o sr o agradecer o contato e informamo ...,"[('prezar', 'o'), ('o', 'sr'), ('sr', 'o'), ('...","[('prezar', 'o', 'sr'), ('o', 'sr', 'o'), ('sr...",0.444821,0.499724,0.634349,...,0.413470,0.404744,0.352644,0.272323,0.292353,0.365345,0.646133,0.497469,0.474224,0.546584
3,"""Prezado (a) Sr. (a), Agradecemos o contato e...",c234,1,"['prezado', 'a', 'sr', 'a', 'agradecemos', 'o'...",prezar o sr o agradecer o contato e informamo ...,"[('prezar', 'o'), ('o', 'sr'), ('sr', 'o'), ('...","[('prezar', 'o', 'sr'), ('o', 'sr', 'o'), ('sr...",0.338893,0.664219,0.520465,...,0.502520,0.439282,0.362558,0.296149,0.379228,0.237873,0.573954,0.540782,0.604582,0.450783
4,"""Prezado Prof. Gilberto Tadeu Reis da Silva ...",c234,1,"['prezado', 'prof', 'gilberto', 'tadeu', 'reis...",prezar prof gilberto tader rei de o Silva em a...,"[('prezar', 'prof'), ('prof', 'gilberto'), ('g...","[('prezar', 'prof', 'gilberto'), ('prof', 'gil...",0.496814,0.420793,0.574704,...,0.602760,0.576759,0.383231,0.367842,0.531815,0.299715,0.646647,0.398757,0.508204,0.586519
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,"Prezado Senhor Thiago Bernardo dos Santos, A...",c5,2,"['prezado', 'senhor', 'thiago', 'bernardo', 'd...",prezar senhor thiago bernardo de o santo o res...,"[('prezar', 'senhor'), ('senhor', 'thiago'), (...","[('prezar', 'senhor', 'thiago'), ('senhor', 't...",0.390466,0.454269,0.683070,...,0.703865,0.481760,0.442732,0.233113,0.396476,0.428575,0.684441,0.308631,0.510142,0.479632
2996,"Prezado Senhor Thyago, O DNPM informa a Vossa...",c1,0,"['prezado', 'senhor', 'thyago', 'o', 'dnpm', '...",prezar senhor thyago o dnpm informar o vosso s...,"[('prezar', 'senhor'), ('senhor', 'thyago'), (...","[('prezar', 'senhor', 'thyago'), ('senhor', 't...",0.349672,0.757337,0.480575,...,0.342588,0.374892,0.794383,0.368405,0.324414,0.527697,0.545426,0.483875,0.615697,0.488009
2997,"Prezado Senhor Tiago Puhlmann, Em atenção ...",c234,1,"['prezado', 'senhor', 'tiago', 'puhlmann', 'em...",prezar senhor tiago puhlmann em atenção a o de...,"[('prezar', 'senhor'), ('senhor', 'tiago'), ('...","[('prezar', 'senhor', 'tiago'), ('senhor', 'ti...",0.371255,0.503759,0.662009,...,0.493813,0.381026,0.265614,0.266990,0.571446,0.370686,0.654652,0.577308,0.654171,0.370949
2998,"Prezado Senhor Tiago, A Lei n&deg, 8001/1990,...",c1,0,"['prezado', 'senhor', 'tiago', 'a', 'lei', 'nd...",prezar senhor tiago o lei ndeg 80011990 altera...,"[('prezar', 'senhor'), ('senhor', 'tiago'), ('...","[('prezar', 'senhor', 'tiago'), ('senhor', 'ti...",0.306295,0.568884,0.512193,...,0.498813,0.488535,0.230530,0.163669,0.654940,0.195518,0.592447,0.512161,0.713361,0.544840


In [75]:
final.head()

,resp_text,clarity,Class,tokens,lemma,ngram2,ngram3,0,1,2,...,758,759,760,761,762,763,764,765,766,767
0,Prezado Sr Jose Taunai Em atenção ao seu pe...,c5,2,"['prezado', 'sr', 'jose', 'taunai', 'em', 'ate...",prezar sr jose taunai em atenção a o seu pedir...,"[('prezar', 'sr'), ('sr', 'jose'), ('jose', 't...","[('prezar', 'sr', 'jose'), ('sr', 'jose', 'tau...",0.366699,0.436405,0.597397,...,0.646290,0.461819,0.457743,0.168369,0.451732,0.390573,0.582483,0.394189,0.513412,0.569378
1,"""A pedido do Pró-Reitor de Graduação, informa...",c5,2,"['a', 'pedido', 'do', 'próreitor', 'de', 'grad...",o pedir de o próreitor de graduação informamo ...,"[('o', 'pedir'), ('pedir', 'de'), ('de', 'o'),...","[('o', 'pedir', 'de'), ('pedir', 'de', 'o'), (...",0.357845,0.643556,0.584826,...,0.540465,0.347499,0.333893,0.188837,0.211751,0.342873,0.642443,0.489474,0.557652,0.561903
2,"""Prezado (a) Sr. (a), Agradecemos o contato e...",c234,1,"['prezado', 'a', 'sr', 'a', 'agradecemos', 'o'...",prezar o sr o agradecer o contato e informamo ...,"[('prezar', 'o'), ('o', 'sr'), ('sr', 'o'), ('...","[('prezar', 'o', 'sr'), ('o', 'sr', 'o'), ('sr...",0.444821,0.499724,0.634349,...,0.413470,0.404744,0.352644,0.272323,0.292353,0.365345,0.646133,0.497469,0.474224,0.546584
3,"""Prezado (a) Sr. (a), Agradecemos o contato e...",c234,1,"['prezado', 'a', 'sr', 'a', 'agradecemos', 'o'...",prezar o sr o agradecer o contato e informamo ...,"[('prezar', 'o'), ('o', 'sr'), ('sr', 'o'), ('...","[('prezar', 'o', 'sr'), ('o', 'sr', 'o'), ('sr...",0.338893,0.664219,0.520465,...,0.502520,0.439282,0.362558,0.296149,0.379228,0.237873,0.573954,0.540782,0.604582,0.450783
4,"""Prezado Prof. Gilberto Tadeu Reis da Silva ...",c234,1,"['prezado', 'prof', 'gilberto', 'tadeu', 'reis...",prezar prof gilberto tader rei de o Silva em a...,"[('prezar', 'prof'), ('prof', 'gilberto'), ('g...","[('prezar', 'prof', 'gilberto'), ('prof', 'gil...",0.496814,0.420793,0.574704,...,0.602760,0.576759,0.383231,0.367842,0.531815,0.299715,0.646647,0.398757,0.508204,0.586519


In [76]:
final.isnull().sum().sum()

0

In [77]:
f1 = final.iloc[0:3000]
f2 = final.iloc[3000:]

In [78]:
f1.to_csv('datasets/BERT_Embedding_1.csv', index=False)
f2.to_csv('datasets/BERT_Embedding_2.csv', index=False)

In [79]:
final.head()

,resp_text,clarity,Class,tokens,lemma,ngram2,ngram3,0,1,2,...,758,759,760,761,762,763,764,765,766,767
0,Prezado Sr Jose Taunai Em atenção ao seu pe...,c5,2,"['prezado', 'sr', 'jose', 'taunai', 'em', 'ate...",prezar sr jose taunai em atenção a o seu pedir...,"[('prezar', 'sr'), ('sr', 'jose'), ('jose', 't...","[('prezar', 'sr', 'jose'), ('sr', 'jose', 'tau...",0.366699,0.436405,0.597397,...,0.646290,0.461819,0.457743,0.168369,0.451732,0.390573,0.582483,0.394189,0.513412,0.569378
1,"""A pedido do Pró-Reitor de Graduação, informa...",c5,2,"['a', 'pedido', 'do', 'próreitor', 'de', 'grad...",o pedir de o próreitor de graduação informamo ...,"[('o', 'pedir'), ('pedir', 'de'), ('de', 'o'),...","[('o', 'pedir', 'de'), ('pedir', 'de', 'o'), (...",0.357845,0.643556,0.584826,...,0.540465,0.347499,0.333893,0.188837,0.211751,0.342873,0.642443,0.489474,0.557652,0.561903
2,"""Prezado (a) Sr. (a), Agradecemos o contato e...",c234,1,"['prezado', 'a', 'sr', 'a', 'agradecemos', 'o'...",prezar o sr o agradecer o contato e informamo ...,"[('prezar', 'o'), ('o', 'sr'), ('sr', 'o'), ('...","[('prezar', 'o', 'sr'), ('o', 'sr', 'o'), ('sr...",0.444821,0.499724,0.634349,...,0.413470,0.404744,0.352644,0.272323,0.292353,0.365345,0.646133,0.497469,0.474224,0.546584
3,"""Prezado (a) Sr. (a), Agradecemos o contato e...",c234,1,"['prezado', 'a', 'sr', 'a', 'agradecemos', 'o'...",prezar o sr o agradecer o contato e informamo ...,"[('prezar', 'o'), ('o', 'sr'), ('sr', 'o'), ('...","[('prezar', 'o', 'sr'), ('o', 'sr', 'o'), ('sr...",0.338893,0.664219,0.520465,...,0.502520,0.439282,0.362558,0.296149,0.379228,0.237873,0.573954,0.540782,0.604582,0.450783
4,"""Prezado Prof. Gilberto Tadeu Reis da Silva ...",c234,1,"['prezado', 'prof', 'gilberto', 'tadeu', 'reis...",prezar prof gilberto tader rei de o Silva em a...,"[('prezar', 'prof'), ('prof', 'gilberto'), ('g...","[('prezar', 'prof', 'gilberto'), ('prof', 'gil...",0.496814,0.420793,0.574704,...,0.602760,0.576759,0.383231,0.367842,0.531815,0.299715,0.646647,0.398757,0.508204,0.586519


In [80]:
df = pd.read_csv('datasets/BERT_Embedding.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'datasets/BERT_Embedding.csv'

In [ ]:
df.describe()

,Class,0,1,2,3,4,5,6,7,8,...,758,759,760,761,762,763,764,765,766,767
count,6000.000000,6000.000000,6000.000000,6000.000000,6000.000000,6000.000000,6000.000000,6000.000000,6000.000000,6000.000000,...,6000.000000,6000.000000,6000.000000,6000.000000,6000.000000,6000.000000,6000.000000,6000.000000,6000.000000,6000.000000
mean,1.000000,0.440475,0.429495,0.497736,0.320519,0.337891,0.366017,0.436796,0.495423,0.541360,...,0.491276,0.396792,0.473437,0.356217,0.446263,0.381357,0.568187,0.398977,0.518481,0.439840
std,0.816565,0.134139,0.131003,0.117983,0.080694,0.086897,0.110354,0.135859,0.139539,0.124830,...,0.136403,0.119263,0.135049,0.091387,0.141306,0.128853,0.127440,0.102243,0.089696,0.136577
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.345082,0.340009,0.419130,0.265811,0.285625,0.295749,0.341638,0.405061,0.458885,...,0.402839,0.311392,0.371460,0.296782,0.350898,0.291557,0.481769,0.333403,0.460380,0.348816
50%,1.000000,0.438957,0.422704,0.503247,0.321416,0.334964,0.356448,0.431753,0.491315,0.538245,...,0.495706,0.394804,0.468844,0.354033,0.443037,0.371979,0.570357,0.400016,0.522596,0.437055
75%,2.000000,0.527806,0.509937,0.577574,0.375962,0.390110,0.430166,0.528237,0.584116,0.621108,...,0.577933,0.471055,0.566584,0.408407,0.534331,0.456043,0.655484,0.461737,0.576530,0.526578
max,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
df.head()

,resp_text,clarity,Class,tokens,lemma,ngram2,ngram3,0,1,2,...,758,759,760,761,762,763,764,765,766,767
0,Prezado Sr Jose Taunai Em atenção ao seu pe...,c5,2,"['prezado', 'sr', 'jose', 'taunai', 'em', 'ate...",prezar sr jose taunai em atenção a o seu pedir...,"[('prezar', 'sr'), ('sr', 'jose'), ('jose', 't...","[('prezar', 'sr', 'jose'), ('sr', 'jose', 'tau...",0.427392,0.212042,0.516438,...,0.584561,0.336162,0.482180,0.273163,0.434368,0.340332,0.491752,0.372363,0.491534,0.550062
1,"""A pedido do Pró-Reitor de Graduação, informa...",c5,2,"['a', 'pedido', 'do', 'próreitor', 'de', 'grad...",o pedir de o próreitor de graduação informamo ...,"[('o', 'pedir'), ('pedir', 'de'), ('de', 'o'),...","[('o', 'pedir', 'de'), ('pedir', 'de', 'o'), (...",0.308308,0.605072,0.438790,...,0.399073,0.327044,0.337716,0.229499,0.230075,0.430749,0.576797,0.456070,0.527956,0.525562
2,"""Prezado (a) Sr. (a), Agradecemos o contato e...",c234,1,"['prezado', 'a', 'sr', 'a', 'agradecemos', 'o'...",prezar o sr o agradecer o contato e informamo ...,"[('prezar', 'o'), ('o', 'sr'), ('sr', 'o'), ('...","[('prezar', 'o', 'sr'), ('o', 'sr', 'o'), ('sr...",0.520456,0.417305,0.493604,...,0.384649,0.294300,0.401364,0.281565,0.367157,0.364539,0.624911,0.454857,0.471248,0.505317
3,"""Prezado (a) Sr. (a), Agradecemos o contato e...",c234,1,"['prezado', 'a', 'sr', 'a', 'agradecemos', 'o'...",prezar o sr o agradecer o contato e informamo ...,"[('prezar', 'o'), ('o', 'sr'), ('sr', 'o'), ('...","[('prezar', 'o', 'sr'), ('o', 'sr', 'o'), ('sr...",0.360526,0.507117,0.409693,...,0.600001,0.231550,0.447652,0.304424,0.460164,0.288831,0.581606,0.369495,0.580037,0.342662
4,"""Prezado Prof. Gilberto Tadeu Reis da Silva ...",c234,1,"['prezado', 'prof', 'gilberto', 'tadeu', 'reis...",prezar prof gilberto tader rei de o Silva em a...,"[('prezar', 'prof'), ('prof', 'gilberto'), ('g...","[('prezar', 'prof', 'gilberto'), ('prof', 'gil...",0.674706,0.244865,0.611399,...,0.583029,0.585966,0.295181,0.552866,0.642015,0.327959,0.653914,0.290733,0.491183,0.564766


In [ ]:
origin = pd.read_excel('datasets/ep1_esic2023_clareza_TRAIN.xlsx')

In [ ]:
origin = origin[origin.duplicated()]

In [ ]:
origin

,resp_text,clarity
9,"""Prezado(a), Em atenção ao seu pedido, en...",c5
71,Análise de mérito realizada no protocolo 9990...,c1
72,Análise de mérito realizada no protocolo 9990...,c1
73,Análise de mérito realizada no protocolo 9990...,c1
74,Análise de mérito realizada no protocolo 9990...,c1
...,...,...
5767,"Senhor, Encaminhamos em anexo resposta ao ped...",c1
5768,"Senhor, Encaminhamos em anexo resposta ao ped...",c1
5769,"Senhor, Encaminhamos em anexo resposta ao ped...",c1
5880,"Senhora Solicitante, Em atenção à sua demand...",c234


In [ ]:
check = final.iloc[71:74]

In [ ]:
check

,resp_text,clarity,Class,tokens,lemma,ngram2,ngram3,0,1,2,...,758,759,760,761,762,763,764,765,766,767
71,Análise de mérito realizada no protocolo 9990...,c1,0,"['análise', 'de', 'mérito', 'realizada', 'no',...",análise de mérito realizar em o protocolo 9990...,"[('análise', 'de'), ('de', 'mérito'), ('mérito...","[('análise', 'de', 'mérito'), ('de', 'mérito',...",0.245976,0.302952,0.361246,...,0.519406,0.571359,0.339145,0.425927,0.366023,0.396791,0.430741,0.544014,0.531847,0.49758
72,Análise de mérito realizada no protocolo 9990...,c1,0,"['análise', 'de', 'mérito', 'realizada', 'no',...",análise de mérito realizar em o protocolo 9990...,"[('análise', 'de'), ('de', 'mérito'), ('mérito...","[('análise', 'de', 'mérito'), ('de', 'mérito',...",0.245976,0.302952,0.361246,...,0.519406,0.571359,0.339145,0.425927,0.366023,0.396791,0.430741,0.544014,0.531847,0.49758
73,Análise de mérito realizada no protocolo 9990...,c1,0,"['análise', 'de', 'mérito', 'realizada', 'no',...",análise de mérito realizar em o protocolo 9990...,"[('análise', 'de'), ('de', 'mérito'), ('mérito...","[('análise', 'de', 'mérito'), ('de', 'mérito',...",0.245976,0.302952,0.361246,...,0.519406,0.571359,0.339145,0.425927,0.366023,0.396791,0.430741,0.544014,0.531847,0.49758
